<a href="https://colab.research.google.com/github/theoVasilas/Diplomatic_incident/blob/CO2/Calculations_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [ ]:
!pip install -q flwr[simulation] flwr-datasets[vision]

In [ ]:
!pip install thop

In [ ]:
from collections import OrderedDict, Counter
from typing import Dict, List, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from flwr_datasets import FederatedDataset


DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")

Training on cpu


# Dataloader


In [ ]:
BATCH_SIZE = 32
NUM_CLIENTS = 10

In [30]:
def load_datasets(partition_id: int, num_partitions: int):
    fds = FederatedDataset(dataset="ylecun/mnist",
                           partitioners={"train": num_partitions})

    pytorch_transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
    def apply_transforms(batch):
        batch["image"] = [pytorch_transforms(img) for img in batch["image"]]
        return batch

    partition = fds.load_partition(partition_id)
    # Divider : 80% train, 20% test
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)
    #Apply Tranforms
    partition_train_test = partition_train_test.with_transform(apply_transforms)

    trainloader = DataLoader(partition_train_test["train"], shuffle=True )
    valloader = DataLoader(partition_train_test["test"])

    testset = fds.load_split("test").with_transform(apply_transforms)
    testloader = DataLoader(testset)

    return trainloader, valloader, testloader

In [31]:
trainloader, valloader, testloader = load_datasets(0,1)

total_size = 0

# Iterate through the images in the partition
for sample in trainloader:
    image = sample["image"]  # Assuming image is a tensor or PIL image
    if isinstance(image, torch.Tensor):
        # If image is a tensor, get the shape and calculate size
        image_size = image.numel() * image.element_size()  # in bytes
    else:
        # If image is a PIL image, get the size in bytes
        image_size = len(image.tobytes())

    total_size += image_size

# Convert the total size to kilobytes
total_size_kb = total_size / 1024.0

print(f"Total size of the partition in bytes: {total_size}")
print(f"Total size of the partition in kilobytes: {total_size_kb}")
print(f"Total size of the partition in Megabytes: {total_size_kb / 1000}")

print(f"{len(trainloader) = }")
print(f"{len(valloader) = }")
print(f"{len(testloader) = }")



Total size of the partition in bytes: 150528000
Total size of the partition in kilobytes: 147000.0
Total size of the partition in Megabytes: 147.0
len(trainloader) = 48000
len(valloader) = 12000
len(testloader) = 10000


# models

In [ ]:
from thop import profile

def Cal_fPO(model):
  input_tensor = torch.randn(1, 1, 28, 28)
  flops, params = profile(model, inputs=(input_tensor,))
  print(f"FLOPs: {flops}")
  print(f"Parameters: {params}")

  trainloader, valloader, _ = load_datasets(0,10)
  datasize = len(trainloader)
  FPO_epoch = 3 * flops * datasize
  FPO_validation = flops * len(valloader)

  print(f"FPO_epoch: {FPO_epoch/10**6}M")
  print(f"FPO_validation: {FPO_validation/10**6}M")

In [ ]:
def cal_param(model):
  num_parameters = sum(value.numel() for value in model.state_dict().values())
  print(f"{num_parameters = }")

  model_size_bytes = num_parameters * 4

  # Convert bytes to megabytes
  model_size_mb = model_size_bytes / (1024 * 1024)

  print(f"num_parameters = {num_parameters}")
  print(f"Model size = {model_size_mb:.2f} MB")

## Model 1

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = Net()
cal_param(net)
Cal_fPO(net)

num_parameters = 44426
num_parameters = 44426
Model size = 0.17 MB
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
FLOPs: 281640.0
Parameters: 44426.0
FPO_epoch: 126.738M
FPO_validation: 10.70232M
